In [1]:
# !pip install transformers
# !pip install git+https://github.com/yuchenlin/LLM-Blender.git
# !pip install "pydantic<2"
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [2]:
import pandas as pd
import numpy as np
import torch
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast, AdamW, AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm
from torch import nn
from transformers import Trainer
import torch.nn.functional as F
import torchvision.transforms as tr # 이미지 전처리 기능들을 제공하는 라이브러리
from torch.utils.data import DataLoader, Dataset # 데이터를 모델에 사용할 수 있도록 정리해 주는 라이브러리
import csv

from llm_blender.pair_ranker.pairrm import DebertaV2PairRM # or copy the DebertaV2PairRM definition here, https://github.com/yuchenlin/LLM-Blender/blob/main/llm_blender/pair_ranker/pairrm.py
from transformers import AutoTokenizer
from typing import List

# CUDA 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

2024-02-13 04:53:33.487723: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-13 04:53:34.587421: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-02-13 04:53:34.587537: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-02-13 04:53:34.587550: W tensorfl


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.8/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/nvidia/lib64')}
  warn(msg)
/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//matplotlib_inline.backend_inline'), PosixPath('module')}
  warn(msg)


Using device: cuda


In [3]:
test_df = pd.read_csv("./data/test.csv")
# test_df

In [4]:
# count = test_df['질문'].str.count('또한')
# count[count>0]

In [5]:

appedix_dict_1 = {}
for idx, row in test_df.iterrows():
    for q_col in ['질문','id']:
        if q_col == '질문':
            if row[q_col].count('?') >=2:
                appendix = row[q_col].split('?')[1] + "?"
                appendix = appendix.strip()
                row[q_col] = row[q_col].split('?')[0] + "?"
                row[q_col] = row[q_col].strip()
                appedix_dict_1[idx] = appendix
            else:
                row[q_col] = row[q_col].strip()
appedix_dict_1   

{0: '또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?',
 11: '또한, 부실 시공으로 인해 타공하자가 발생할 가능성이 있나요?',
 16: '그리고 유성페인트를 사용하는 것에 대한 부작용이 있을까요?',
 17: '그리고 반점이 생긴지 1년 이내인 하자에 대해 어떤 보수작업을 해야 하나요?',
 20: '또한, 피스 하자가 무엇인지 자세히 설명해 주실 수 있나요?',
 22: '또한, 겨울에 도배를 할 때 특별히 신경써야 할 사항이 있을까요?',
 23: '또한, 페인트가 남으면 어디에 보관하는 게 좋을까요?',
 24: '또한, 유광 자기질 타일은 얼마나 오랫동안 사용할 수 있을까요?',
 25: '그리고 스탠드조명을 선택할 때 주의할 점이 있을까요?',
 26: '그리고 불량 도배지를 사용할 경우 도배지가 얼마나 오랫동안 버틸까요?',
 27: '그리고 실크벽지의 교체 주기는 얼마나인가요?',
 28: '그리고 철골구조의 장점은 무엇인가요?',
 29: '그리고 조적식 구조란 무엇인가요?',
 33: '또한, 액체방수공사는 무엇을 하는 것인가요?',
 35: '그리고 장판이 남을 때 어떻게 처리해야 하나요?',
 38: '또한, 콘크리트 벽에 구멍을 뚫는 방법에는 어떤 도구나 기술을 사용해야 하나요?',
 39: '그리고 인테리어에 가장 많이 사용되는 도배재료는 무엇인가요?',
 42: '그리고 복도나 협소한 공간을 확장시키기 위해 가장 효과적인 방법이 무엇일까요?',
 48: '또한, 배관공사 시 통기구를 설치해야 하는 이유가 무엇인가요?',
 49: '또한, 규산질계 침투성 도포 방수공사는 어떤 방식으로 이루어지나요?',
 50: '또한, 내부와 외부 온도의 큰 차이로 인해 곰팡이 발생이 빨라지나요?',
 54: '그리고 기둥-보 구조 방식은 무엇을 의미하나요?',
 56: '그리고 새집증후군을 예방하는 데 가장 효과적인 방법은 무엇인가요?',
 59: '또한, 옥상 방수용 탄성 에멀전 페인트를 사용하는 장점은 무엇인가요?',
 63: 

In [6]:

appedix_dict_2 = {}
for idx, row in test_df.iterrows():
    for q_col in ['질문','id']:
        if q_col == '질문':
            if row[q_col].count('그리고') > 0 and row[q_col].count('이를') == 0 :
                appendix = row[q_col].split('그리고')[1]
                appendix = appendix.strip()
                row[q_col] = row[q_col].split('그리고')[0]
                row[q_col] = row[q_col].strip()
                appedix_dict_2[idx] = appendix
            else:
                row[q_col] = row[q_col].strip()
appedix_dict_2     

{6: '합지벽지의 어떤 단점이 있나요?',
 15: '페인트 상도재의 역할과 사용 방법에 대해 알려주세요.',
 47: '습도가 높을 때 곰팡이가 어떻게 발생하는지 자세히 알고 싶습니다.',
 55: '도배지가 남으면 어떻게 처리해야 하나요?',
 64: '소화기 종류에는 어떤 것들이 있는지 알려주세요.',
 86: '토목이 무엇인지 설명해줘.',
 93: '셀룰로오스의 단점에 대해 설명해주세요.',
 94: '반려동물을 위한 바닥재에는 어떤 종류가 있는지 알려주세요.',
 98: '밀풀 사용 시 주의할 점은 무엇인가요?',
 99: '통나무구조 방식은 어떤 건물에 사용되는 건축 구조 방식인가요?',
 108: '펫테리어가 무엇인지 자세히 알려주세요.',
 119: '철근콘크리트 구조에 대해 좀 더 자세히 알려주세요.'}

In [7]:
appedix_dict_3 = {}
for idx, row in test_df.iterrows():
    for q_col in ['질문','id']:
        if q_col == '질문':
            if row[q_col].count('또한') > 0:
                appendix = row[q_col].split('또한')[1] + "또한"
                appendix = appendix.strip()
                row[q_col] = row[q_col].split('또한')[0] + "또한"
                row[q_col] = row[q_col].strip()
                appedix_dict_3[idx] = appendix
            else:
                row[q_col] = row[q_col].strip()

appedix_dict_3

{2: ', 압출법 단열판을 사용하는 것의 장점은 무엇인가요?또한',
 32: ', 세라믹 타일을 사용할 때 고려해야 할 단점은 무엇인가요?또한',
 45: ', 낡은 목재 가구의 흠집을 숨기는 방법을 알려주세요.또한',
 52: ', 속건형 유성 발수제의 사용 목적과 효과에 대해 알려주세요.또한',
 72: ', 강화마루의 장점은 무엇입니까?또한',
 89: ', 아이소핑크의 장점은 무엇인가요?또한'}

In [8]:
test_df['질문']

for i in range(len(test_df)):
    print(test_df['질문'][i])


방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요?
도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
큐블럭의 단점을 알려주세요. 또한
철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있을까요?
도배지의 완전한 건조를 위해 몇 주 동안 기다려야 하나요?
철근철골콘크리트 구조가 적용된 건물의 장단점은 무엇인가요?
어떤 상황에 개별 공간이 더 적합한지, 어떤 상황에 오픈 플랜 공간이 더 적합한지 알려주세요.
도배지 들뜸 현상이 발생하는 가장 일반적인 원인은 무엇인가요?
도배지에 얼룩이 생기는 다양한 원인들에 대해서 자세히 알려주세요.
고층 건물을 건축할 때 철골구조가 주로 선택되는 이유는 무엇인가요?
유성페인트의 환경 오염에 대한 예시를 알려주세요.
훼손과 오염은 어떻게 다른가요?
철골콘크리트 구조에서 사용되는 철근의 재료적 특징은 무엇인가요?"
방염벽지가 실크벽지와 합지벽지와 다른 점은 무엇인가요?
도배지에 곰팡이가 생겼을 때 높은 습도가 원인인 경우, 습기 관리는 어떻게 해야 할까요?"
롱브릭타일은 어떤 종류의 타일인가요?
준불연재료는 무엇인가요?
어떤 환경 요인이 몰딩 수정을 유발할 수 있는가요?
벽에 뚫린 구멍이 이상하다고 하셨는데, 타공 불량이 발생하는 원인과 그 해결 방법에 대해 자세히 설명해주실 수 있을까요?"
다이닝 룸을 더 고급스럽게 꾸밀려면 어떻게 해야 하나요와 아레카 야자를 키우면 어떤 이점이 있는지의 관계는 무엇인가요?
습도가 높아서 석고수정이 발생하나요?
공동주택의 실내 공기질을 측정할 때, 어떤 층에서 샘플을 채취하는 것이 가장 적절한가요?"
타일 바닥에서 파손된 타일을 교체하는 과정에 어떤 단계가 포함되나요?
유성페인트를 사용하는 것에 대한 부작용이 있을까요?
가장 비싼 바닥재는 어떤 종류인가요?
풍수지리를 활용하여 집을 꾸밀 때 어떤 주의사항을 고려해야 할까요?
반점이 1년 이상 생긴 하자를 보수하는 방법은 무엇인가요?
평지붕의 단점은 무엇인가요?
경량철골구조는 어떤 건물에 

In [9]:

                  
tokenizer = PreTrainedTokenizerFast.from_pretrained('./skt_aug3_4_epoch/', eos_token='</s>')
model = GPT2LMHeadModel.from_pretrained("./skt_aug3_4_epoch/")
model.to(device)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [10]:

# Inference를 위한 test.csv 파일 로드
# test = pd.read_csv('./data/test.csv')

# test.csv의 '질문'에 대한 '답변'을 저장할 리스트
file_path = "result_skt_r4.txt"
total_preds = []
# '질문' 컬럼의 각 질문에 대해 답변 생성
# for idx,model in enumerate(model_list):
preds = []
idx = 0
for test_question in tqdm(test_df['질문']):
    # 입력 텍스트를 토큰화하고 모델 입력 형태로 변환
    
    input_ids = tokenizer.encode(test_question + tokenizer.eos_token, return_tensors='pt')

    # 답변 생성
    with torch.no_grad():

        # output_sequences = model.generate(
        #     input_ids=input_ids.to(device),
        #     max_length=2000,
        #     min_length=20,
        #     temperature=1.5,
        #     top_k=120,
        #     top_p=1.5,
        #     repetition_penalty=1.0,
        #     do_sample=True,
        #     num_beams=120,
        #     # no_repeat_ngram_size=3,
        #     # num_return_sequences=1,
        #     # eos_token_id=tokenizer.eos_token_id
        # )
        output_sequences = model.generate(
            input_ids=input_ids.to(device),
            max_length=2000,
            min_length=20,
            temperature=0.8,
            top_k=150,
            top_p=0.8,
            repetition_penalty=1.0,
            do_sample=True,
            num_beams=150,
            # no_repeat_ngram_size=3,
            # num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id
        )

    for generated_sequence in output_sequences:
        full_text = tokenizer.decode(generated_sequence, skip_special_tokens=False)
        print(full_text)
        answer_start = full_text.find(tokenizer.eos_token) 
        anser_end = full_text.rfind(tokenizer.eos_token) 
        answer_only = full_text[answer_start+5:anser_end].strip()
        answer_only = answer_only.replace('\n', ' ')
        print(answer_only)
        preds.append(answer_only)
        answer_only = answer_only+'\n'
    if idx==0:
        with open(file_path, "w") as file:
            file.write(answer_only)
    else:
        with open(file_path, "a") as file:
            file.write(answer_only)
    idx+=1


    # with open(csv_file_path, mode='w') as csv_file:
    #     csv_writer = csv.writer(csv_file)
    #     csv_writer.writerow(answer_only)
# break
# total_preds.append(preds)
# model = model.cpu()
# torch.cuda.empty_cache()

# 원목마루와 롱브릭타일에 대해 설명해주세요. 각각의 단점과 특징은 무엇인가요?


  1%|█▎                                                                                                                                                                         | 1/130 [00:03<07:57,  3.70s/it]

방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요?</s> 방청 페인트의 종류로는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 그리고 규산염페인트 등이 있습니다. 이러한 다양한 종류의 방청 페인트는 각자의 특징과 용도에 따라 선택되어 사용됩니다.</s>
방청 페인트의 종류로는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 그리고 규산염페인트 등이 있습니다. 이러한 다양한 종류의 방청 페인트는 각자의 특징과 용도에 따라 선택되어 사용됩니다.


  2%|██▋                                                                                                                                                                        | 2/130 [00:12<14:36,  6.85s/it]

도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?</s> 도배지에 녹이 묻어나오는 것은 녹오염으로, 도배지 상에 녹색 또는 갈색의 녹이 발생하는 현상을 의미합니다. 이러한 현상이 발생하는 원인, 책임 소재 및 해결 방법은 다음과 같습니다.  1. 습도 문제 원인: 높은 습도로 인해 도배지 안쪽의 금속 물질이 산화되어 녹이 도배지에 발생할 수 있습니다. 책임 소재: 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법: 제습기를 가동하거나 실내의 적정 습도를 유지하기 위해 환기를 통해 습기를 제거하는 것이 중요합니다. 전문가의 도움을 받아 보수 작업을 권장합니다.  2. 누수 원인: 누수로 도배지가 젖어 있는 상태가 계속되면 곰팡이와 녹이 발생할 수 있습니다. 책임 소재: 누수 문제는 건물의 소유자나 거주자의 관리 책임이 있습니다. 해결 방법: 누수를 해결하고 곰팡이와 녹이 발생한 부분 및 전체 도배지를 교체하는 보수 작업이 필요합니다. 이러한 작업은 전문가의 도움을 받는 것이 좋습니다.</s>
도배지에 녹이 묻어나오는 것은 녹오염으로, 도배지 상에 녹색 또는 갈색의 녹이 발생하는 현상을 의미합니다. 이러한 현상이 발생하는 원인, 책임 소재 및 해결 방법은 다음과 같습니다.  1. 습도 문제 원인: 높은 습도로 인해 도배지 안쪽의 금속 물질이 산화되어 녹이 도배지에 발생할 수 있습니다. 책임 소재: 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법: 제습기를 가동하거나 실내의 적정 습도를 유지하기 위해 환기를 통해 습기를 제거하는 것이 중요합니다. 전문가의 도움을 받아 보수 작업을 권장합니다.  2. 누수 원인: 누수로 도배지가 젖어 있는 상태가 계속되면 곰팡이와 녹이 발생할 수 있습니다. 책임 소재: 누수 문제는 건물의 소유자나 거주자의 관리 책임이 있습니다. 해결 방법: 누수를 해결하고 곰팡이와 녹이 발생한 부분 및 전체 도배지를 교체하는 보수 작업이 필요합니다. 이러한 작업은 전문가의 도움을 받는 것이 좋습

  2%|███▉                                                                                                                                                                       | 3/130 [00:14<09:28,  4.47s/it]

...

평지붕의 누수 문제를 방지하기 위해 수성 벽체용 탄성 방수 도료를 사용하는 것이 어떤 장점이 있나요?</s> 평지붕의 누수를 방지하기 위해서는 먼저, 누수가 발생하는 부분을 확인합니다. 수도계량기를 확인하는 것입니다. 실내의 모든 수도꼭지를 잠갔는데도 수도계량기의 별표가 돌아간다면 누수가 발생한다는 것이니 전문가와 상담해주세요. 또한, 누수가 발생하는 위치에 따라 물단열판 또는 열화상카메라 등의 전문 장비를 사용하여 누수를 진단할 수도 있습니다. 하지만 누수의 정확한 위치를 파악하는 일은 전문가의 도움을 받는 것이 좋습니다. 만약 누수가 의심된다면 전문가와 상담하여 신속한 조치를 취하는 것이 중요합니다.</s>
평지붕의 누수를 방지하기 위해서는 먼저, 누수가 발생하는 부분을 확인합니다. 수도계량기를 확인하는 것입니다. 실내의 모든 수도꼭지를 잠갔는데도 수도계량기의 별표가 돌아간다면 누수가 발생한다는 것이니 전문가와 상담해주세요. 또한, 누수가 발생하는 위치에 따라 물단열판 또는 열화상카메라 등의 전문 장비를 사용하여 누수를 진단할 수도 있습니다. 하지만 누수의 정확한 위치를 파악하는 일은 전문가의 도움을 받는 것이 좋습니다. 만약 누수가 의심된다면 전문가와 상담하여 신속한 조치를 취하는 것이 중요합니다.


 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋ | 129/130 [06:17<00:05,  5.93s/it]

석고수정이 발생하는 가장 큰 원인은 무엇인가요? 그리고 이를 해결하는 방법에 대해 알려주세요."</s> 석고수정은 벽을 이루고 있는 석고에 손상 또는 변형이 발생하는 현상을 말합니다. 석고수정이 발생하는 주요 원인과 책임소재, 그리고 해결 방법은 다음과 같습니다.  1. 건축 물리적 변화    - 원인: 건물의 노후화나 지진과 같은 물리적 변화로 벽을 이루고 있는 석고가 수정될 수 있습니다.    - 책임소재: 건물의 소유자나 거주자가 책임을 집니다.    - 해결 방법: 수정된 석고를 복구하고, 전문가에게 정밀한 진단을 의뢰해야 합니다.  2. 공간 내 높은 습도    - 원인: 습기에 노출되면 석고가 변형될 수 있습니다.    - 책임소재: 건물의 소유자나 거주자가 습기 관리에 책임이 있습니다.    - 해결 방법: 제습기를 가동하거나 환기를 통해 적정한 실내 습도를 유지하고, 석고를 수정하는 전문가의 도움을 받아야 합니다.  3. 구조적 결함    - 원인: 건물 자체의 구조적 결함으로 석고 수정이 발생할 수 있습니다.    - 책임소재: 건물의 소유자나 거주자가 책임이 있습니다.    - 해결 방법: 건물의 구조적 결함을 해결하고, 수정된 석고를 전문가의 도움을 받아 복구하는 것이 필요합니다.그리고아이가 있는 집에는 두꺼운 단열재를 사용하는 것이 좋습니다. 아이가 있는 공간은 층간 소음이 발생할 가능성이 크며, 뿐만 아니라 음료 등을 쏟을 위험 또한 존재합니다. 이에 두꺼운 단열재는 소음을 줄이는 효과가 있을 뿐만 아니라 오염을 제거하기 용이하여 유용합니다. 그러므로 아이가 있는 집에서는 두꺼운 단열재를 사용하는 것이 바람직합니다.</s>
석고수정은 벽을 이루고 있는 석고에 손상 또는 변형이 발생하는 현상을 말합니다. 석고수정이 발생하는 주요 원인과 책임소재, 그리고 해결 방법은 다음과 같습니다.  1. 건축 물리적 변화    - 원인: 건물의 노후화나 지진과 같은 물리적 변화로 벽을 이루고 있는 석고가 수정될 수 있습니다.    - 책임소재: 건물의 

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 130/130 [06:19<00:00,  2.92s/it]

카페트의 기대 수명은 얼마나 될까요?</s> 카페트의 기대수명은 일반적으로 6년입니다. 다만, 사용 빈도, 청소 방식, 햇빛 노출 정도 등 사용 환경과 관리에 따라 수명이 달라질 수 있으므로 주기적으로 청소 및 관리를 해야 합니다. 또한, 질 좋은 카페트를 구매하고 깔끔하게 관리하여 수명을 연장할 수 있습니다.</s>
카페트의 기대수명은 일반적으로 6년입니다. 다만, 사용 빈도, 청소 방식, 햇빛 노출 정도 등 사용 환경과 관리에 따라 수명이 달라질 수 있으므로 주기적으로 청소 및 관리를 해야 합니다. 또한, 질 좋은 카페트를 구매하고 깔끔하게 관리하여 수명을 연장할 수 있습니다.


In [11]:
file_path = "result_skt_appendix_1_r4.txt"


for idx, test_question in tqdm(appedix_dict_1.items()):
    # 입력 텍스트를 토큰화하고 모델 입력 형태로 변환
    
    input_ids = tokenizer.encode(test_question + tokenizer.eos_token, return_tensors='pt')

    # 답변 생성
    with torch.no_grad():

        # output_sequences = model.generate(
        #     input_ids=input_ids.to(device),
        #     max_length=2000,
        #     min_length=20,
        #     temperature=1.5,
        #     top_k=120,
        #     top_p=1.5,
        #     repetition_penalty=1.0,
        #     do_sample=True,
        #     num_beams=120,
        #     # no_repeat_ngram_size=3,
        #     # num_return_sequences=1,
        #     # eos_token_id=tokenizer.eos_token_id
        # )
        output_sequences = model.generate(
            input_ids=input_ids.to(device),
            max_length=2000,
            min_length=20,
            temperature=0.8,
            top_k=150,
            top_p=0.8,
            repetition_penalty=1.0,
            do_sample=True,
            num_beams=150,
            # no_repeat_ngram_size=3,
            # num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id
        )

    for generated_sequence in output_sequences:
        full_text = tokenizer.decode(generated_sequence, skip_special_tokens=False)
        print(full_text)
        answer_start = full_text.find(tokenizer.eos_token) 
        anser_end = full_text.rfind(tokenizer.eos_token) 
        answer_only = full_text[answer_start+5:anser_end].strip()
        answer_only = answer_only.replace('\n', ' ')
        preds.append(answer_only)
        answer_only = str(idx)+answer_only+'\n'
    if idx==0:
        with open(file_path, "w") as file:
            file.write(answer_only)
    else:
        with open(file_path, "a") as file:
            file.write(answer_only)
    idx+=1

  2%|███▋                                                                                                                                                                        | 1/46 [00:01<00:48,  1.08s/it]

또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?</s> 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.</s>


  4%|███████▍                                                                                                                                                                    | 2/46 [00:01<00:42,  1.02it/s]

또한, 부실 시공으로 인해 타공하자가 발생할 가능성이 있나요?</s> 적절하지 않은 도구로 작업하거나 타공부위의 크기를 정확히 측정하지 않고 작업하면 타공불량이 발생할 수 있습니다.</s>


  7%|███████████▏                                                                                                                                                                | 3/46 [00:07<02:01,  2.83s/it]

...

그리고 내진구조란 무엇인가요?</s> 내진구조란 지진이 발생했을 때 건물이나 시설물이 안전하게 유지되도록 설계된 구조를 말합니다. 이는 지진에 대한 내구성을 갖추기 위해 철근 콘크리트와 같은 내진벽과 다양한 부재를 사용하여 건물을 견고하게 만드는 것을 포함합니다. 내진구조는 지진 발생 시 인명 피해와 재산 손실을 최소화하는 데 중요한 역할을 합니다.</s>


 98%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎   | 45/46 [01:54<00:01,  1.48s/it]

그리고 아파트 도배 평수를 어떻게 계산해야 하나요?</s> 아파트 도배평수는 일반적으로 분양 평수*2.5로 계산합니다.</s>


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 46/46 [01:55<00:00,  2.52s/it]

그리고 오리지널징크의 장점에는 무엇이 있나요?</s> 오리지널징크는 다른 징크에 비해 수명이 길고 다양한 패턴과 디자인이 가능하며 친환경적이고 금속 부식에 대한 내식성이 뛰어나 유지보수가 용이하다는 장점이 있습니다.</s>


In [12]:
file_path = "result_skt_appendix_2_r4.txt"


for idx, test_question in tqdm(appedix_dict_2.items()):
    # 입력 텍스트를 토큰화하고 모델 입력 형태로 변환
    
    input_ids = tokenizer.encode(test_question + tokenizer.eos_token, return_tensors='pt')

    # 답변 생성
    with torch.no_grad():

        # output_sequences = model.generate(
        #     input_ids=input_ids.to(device),
        #     max_length=2000,
        #     min_length=20,
        #     temperature=1.5,
        #     top_k=120,
        #     top_p=1.5,
        #     repetition_penalty=1.0,
        #     do_sample=True,
        #     num_beams=120,
        #     # no_repeat_ngram_size=3,
        #     # num_return_sequences=1,
        #     # eos_token_id=tokenizer.eos_token_id
        # )
        output_sequences = model.generate(
            input_ids=input_ids.to(device),
            max_length=2000,
            min_length=20,
            temperature=0.8,
            top_k=150,
            top_p=0.8,
            repetition_penalty=1.0,
            do_sample=True,
            num_beams=150,
            # no_repeat_ngram_size=3,
            # num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id
        )

    for generated_sequence in output_sequences:
        full_text = tokenizer.decode(generated_sequence, skip_special_tokens=False)
        print(full_text)
        answer_start = full_text.find(tokenizer.eos_token) 
        anser_end = full_text.rfind(tokenizer.eos_token) 
        answer_only = full_text[answer_start+5:anser_end].strip()
        answer_only = answer_only.replace('\n', ' ')
        preds.append(answer_only)
        answer_only = str(idx)+answer_only+'\n'
    if idx==0:
        with open(file_path, "w") as file:
            file.write(answer_only)
    else:
        with open(file_path, "a") as file:
            file.write(answer_only)
    idx+=1

  8%|██████████████▎                                                                                                                                                             | 1/12 [00:01<00:12,  1.15s/it]

합지벽지의 어떤 단점이 있나요?</s> 합지벽지의 단점은 상대적으로 수명이 짧고 내구성이 약하며 오염이 생기면 닦거나 청소가 어렵고 시간이 지나면 변색이 될 수 있는 점이 단점입니다.</s>


 17%|████████████████████████████▋                                                                                                                                               | 2/12 [00:02<00:15,  1.54s/it]

페인트 상도재의 역할과 사용 방법에 대해 알려주세요.</s> 페인트 상도재는 최종 마감으로 칠하는 페인트로, 영어로는 Top 코팅이라고 합니다. 주요 역할은 하도재와 주도재를 보호하고, 외부 표면에 노출되는 것으로부터 제품을 보호하는 것입니다. 이러한 특성으로 열과 기후에 강하며 내구성이 높지만, 가격이 다소 비쌀 수 있습니다.</s>


 25%|███████████████████████████████████████████                                                                                                                                 | 3/12 [00:04<00:15,  1.76s/it]

...

통나무구조 방식은 어떤 건물에 사용되는 건축 구조 방식인가요?</s> 통나무구조 방식은 원형 또는 각형의 내력벽으로 하고 나머지 바닥이나 지붕구조는 2x4경량 목구조와 동일한 구조로 구성된다. 벽체가 통나무를 쌓는 구조이기 때문에 1개층 높이에서 7~12cm의 침하가 장기적으로 발생하게 되므로 창문틀 등의 개구부에는 침하를 고려한 디테일을 만들어야 합니다.</s>


 92%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊              | 11/12 [00:17<00:01,  1.69s/it]

펫테리어가 무엇인지 자세히 알려주세요.</s> 펫테리어란 반려동물을 뜻하는 펫과 인테리어의 합성어를 의미합니다. 반려동물과의 생활을 고려한 편의성을 높이면서도 주거 공간의 인테리어도 고려한 인테리어입니다.</s>


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:19<00:00,  1.63s/it]

철근콘크리트 구조에 대해 좀 더 자세히 알려주세요.</s> 철근콘크리트 구조는 콘크리트의 압축 강도를 살려내는 동시에 철근으로 인장 강도를 보완하여 구축된 구조체입니다. 일반적으로 도심에서 많이 사용되며 내구성, 내화 및 내진 성능이 우수합니다. 그러나 고층 건물에서는 아래층의 보나 기둥이 상당한 무게를 지탱해야 하므로 구조 요소의 단면이 커야 하는 문제가 있습니다.</s>


In [13]:
file_path = "result_skt_appendix_2_r4.txt"


for idx, test_question in tqdm(appedix_dict_2.items()):
    # 입력 텍스트를 토큰화하고 모델 입력 형태로 변환
    
    input_ids = tokenizer.encode(test_question + tokenizer.eos_token, return_tensors='pt')

    # 답변 생성
    with torch.no_grad():

        # output_sequences = model.generate(
        #     input_ids=input_ids.to(device),
        #     max_length=2000,
        #     min_length=20,
        #     temperature=1.5,
        #     top_k=120,
        #     top_p=1.5,
        #     repetition_penalty=1.0,
        #     do_sample=True,
        #     num_beams=120,
        #     # no_repeat_ngram_size=3,
        #     # num_return_sequences=1,
        #     # eos_token_id=tokenizer.eos_token_id
        # )
        output_sequences = model.generate(
            input_ids=input_ids.to(device),
            max_length=2000,
            min_length=20,
            temperature=0.8,
            top_k=150,
            top_p=0.8,
            repetition_penalty=1.0,
            do_sample=True,
            num_beams=150,
            # no_repeat_ngram_size=3,
            # num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id
        )

    for generated_sequence in output_sequences:
        full_text = tokenizer.decode(generated_sequence, skip_special_tokens=False)
        print(full_text)
        answer_start = full_text.find(tokenizer.eos_token) 
        anser_end = full_text.rfind(tokenizer.eos_token) 
        answer_only = full_text[answer_start+5:anser_end].strip()
        answer_only = answer_only.replace('\n', ' ')
        preds.append(answer_only)
        answer_only = str(idx)+answer_only+'\n'
    if idx==0:
        with open(file_path, "w") as file:
            file.write(answer_only)
    else:
        with open(file_path, "a") as file:
            file.write(answer_only)
    idx+=1

  8%|██████████████▎                                                                                                                                                             | 1/12 [00:01<00:11,  1.07s/it]

합지벽지의 어떤 단점이 있나요?</s> 합지벽지의 단점은 상대적으로 수명이 짧고 내구성이 약하며 오염이 생기면 닦거나 청소가 어렵고 시간이 지나면 변색이 될 수 있는 점이 단점입니다.</s>


 17%|████████████████████████████▋                                                                                                                                               | 2/12 [00:02<00:14,  1.48s/it]

페인트 상도재의 역할과 사용 방법에 대해 알려주세요.</s> 페인트 상도재는 최종 마감으로 칠하는 페인트로, 영어로는 Top 코팅이라고 합니다. 주요 역할은 하도재와 주도재를 보호하고, 외부 표면에 노출되는 것으로부터 제품을 보호하는 것입니다. 이러한 특성으로 열과 기후에 강하며 내구성이 높지만, 가격이 다소 비쌀 수 있습니다.</s>


 25%|███████████████████████████████████████████                                                                                                                                 | 3/12 [00:04<00:15,  1.69s/it]

...

통나무구조 방식은 어떤 건물에 사용되는 건축 구조 방식인가요?</s> 통나무구조 방식은 원형 또는 각형의 내력벽으로 하고 나머지 바닥이나 지붕구조는 2x4경량 목구조와 동일한 구조로 구성된다. 벽체가 통나무를 쌓는 구조이기 때문에 1개층 높이에서 7~12cm의 침하가 장기적으로 발생하게 되므로 창문틀 등의 개구부에는 침하를 고려한 디테일을 만들어야 합니다.</s>


 92%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊              | 11/12 [00:17<00:01,  1.65s/it]

펫테리어가 무엇인지 자세히 알려주세요.</s> 펫테리어란 반려동물을 뜻하는 펫과 인테리어의 합성어를 의미합니다. 반려동물과의 생활을 고려한 편의성을 높이면서도 주거 공간의 인테리어도 고려한 인테리어입니다.</s>


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:19<00:00,  1.60s/it]

철근콘크리트 구조에 대해 좀 더 자세히 알려주세요.</s> 철근콘크리트 구조는 콘크리트의 압축 강도를 살려내는 동시에 철근으로 인장 강도를 보완하여 구축된 구조체입니다. 일반적으로 도심에서 많이 사용되며 내구성, 내화 및 내진 성능이 우수합니다. 그러나 고층 건물에서는 아래층의 보나 기둥이 상당한 무게를 지탱해야 하므로 구조 요소의 단면이 커야 하는 문제가 있습니다.</s>


In [14]:
file_path = "result_skt_appendix_3_r4.txt"


for idx, test_question in tqdm(appedix_dict_3.items()):
    # 입력 텍스트를 토큰화하고 모델 입력 형태로 변환
    
    input_ids = tokenizer.encode(test_question + tokenizer.eos_token, return_tensors='pt')

    # 답변 생성
    with torch.no_grad():

        # output_sequences = model.generate(
        #     input_ids=input_ids.to(device),
        #     max_length=2000,
        #     min_length=20,
        #     temperature=1.5,
        #     top_k=120,
        #     top_p=1.5,
        #     repetition_penalty=1.0,
        #     do_sample=True,
        #     num_beams=120,
        #     # no_repeat_ngram_size=3,
        #     # num_return_sequences=1,
        #     # eos_token_id=tokenizer.eos_token_id
        # )
        output_sequences = model.generate(
            input_ids=input_ids.to(device),
            max_length=2000,
            min_length=20,
            temperature=0.8,
            top_k=150,
            top_p=0.8,
            repetition_penalty=1.0,
            do_sample=True,
            num_beams=150,
            # no_repeat_ngram_size=3,
            # num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id
        )
        # output_sequences = model.generate(
        #     input_ids=input_ids.to(device),
        #     max_length=2000,
        #     min_length=20,
        #     temperature=1.2,
        #     top_k=150,
        #     top_p=1.2,
        #     repetition_penalty=1.4,
        #     do_sample=True,
        #     num_beams=300,
        #     no_repeat_ngram_size=6,
        #     # num_return_sequences=1,
        #     eos_token_id=tokenizer.eos_token_id
        # )

    for generated_sequence in output_sequences:
        full_text = tokenizer.decode(generated_sequence, skip_special_tokens=False)
        print(full_text)
        answer_start = full_text.find(tokenizer.eos_token) 
        anser_end = full_text.rfind(tokenizer.eos_token) 
        answer_only = full_text[answer_start+5:anser_end].strip()
        answer_only = answer_only.replace('\n', ' ')
        preds.append(answer_only)
        answer_only = str(idx)+answer_only+'\n'
    if idx==0:
        with open(file_path, "w") as file:
            file.write(answer_only)
    else:
        with open(file_path, "a") as file:
            file.write(answer_only)
    idx+=1

 17%|████████████████████████████▊                                                                                                                                                | 1/6 [00:01<00:07,  1.58s/it]

, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?또한</s> 압출법 단열판은 습기에 강하고, 곰팡이 및 세균의 증식을 방지하기 때문에 실내 환경을 개선할 수 있습니다. 또한, 열전도율이 매우 낮아 효율적인 단열 효과를 제공하며 시공이 간편하여 작업 시간을 단축할 수 있는 장점이 있습니다.</s>


 33%|█████████████████████████████████████████████████████████▋                                                                                                                   | 2/6 [00:03<00:06,  1.67s/it]

, 세라믹 타일을 사용할 때 고려해야 할 단점은 무엇인가요?또한</s> 세라믹 타일의 단점은 저온으로 구워지다 보니 내장용으로만 사용해야 하며, 경도가 낮아 충격과 마모에 취약하다는 점입니다. 더불어 큰 크기로 제작하기 어렵고, 표면이 미끄러워 삼림 지역이나 욕실 등에서의 사용에 적합하지 않다는 단점이 있을 수 있습니다.</s>


 50%|██████████████████████████████████████████████████████████████████████████████████████▌                                                                                      | 3/6 [00:04<00:03,  1.27s/it]

...

, 속건형 유성 발수제의 사용 목적과 효과에 대해 알려주세요.또한</s> 속건형 유성 발수제는 유성계 발수제로서 도장 후 용제가 증발되면서 발수효과를 발휘하는 제품입니다. 이 제품은 흔히 자동차나 창고 등에서 사용되며, 표면에 발라지면 용제가 증발하여 수분이 잘 떨어지는 효과를 내어 물을 효과적으로 방지하는 역할을 합니다. 그러므로 특히 수분에 노출되는 환경에서 사용하기에 적합한 발수제로 폭넓게 활용되고 있습니다.</s>


 83%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                            | 5/6 [00:08<00:01,  1.79s/it]

, 강화마루의 장점은 무엇입니까?또한</s> 강화마루는 저렴한 가격과 시공비로 지은 장점이 있습니다. 또한, 강화마루는 충격과 흠집에 강한 속성을 가지고 있어서 내구성이 좋습니다. 또한 시공, 유지보수, 그리고 철거가 용이하며, 접착 시공 없이 설치할 수 있어서 친환경적이라는 점도 장점으로 꼽힙니다.</s>


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:09<00:00,  1.59s/it]

, 아이소핑크의 장점은 무엇인가요?또한</s> 아이소핑크는 습기에 강하고 곰팡이 및 세균 증식을 막을 수 있으며 단열재 중 열전도율이 가장 낮은 편이고 시공이 간편하다는 장점이 있습니다.</s>


In [15]:


# source_prefix = "<|source|>"
# cand1_prefix = "<|candidate1|>"
# cand2_prefix = "<|candidate2|>"
# inputs = ["hello!", "I love you!"]
# candidates_A = ["hi!", "I hate you!"]
# candidates_B = ["f**k off!", "I love you, too!"]
# def tokenize_pair(sources:List[str], candidate1s:List[str], candidate2s:List[str], source_max_length=1224, candidate_max_length=412):
#     ids = []
#     assert len(sources) == len(candidate1s) == len(candidate2s)
#     max_length = source_max_length + 2 * candidate_max_length
#     for i in range(len(sources)):
#         source_ids = tokenizer.encode(source_prefix + sources[i], max_length=source_max_length, truncation=True)
#         candidate_max_length = (max_length - len(source_ids)) // 2
#         candidate1_ids = tokenizer.encode(cand1_prefix + candidate1s[i], max_length=candidate_max_length, truncation=True)
#         candidate2_ids = tokenizer.encode(cand2_prefix + candidate2s[i], max_length=candidate_max_length, truncation=True)
#         ids.append(source_ids + candidate1_ids + candidate2_ids)
#     encodings = tokenizer.pad({"input_ids": ids}, return_tensors="pt", padding="max_length", max_length=max_length)
#     return encodings

# encodings = tokenize_pair(inputs, candidates_A, candidates_B)
# encodings = {k:v.to(pairrm.device) for k,v in encodings.items()}
# outputs = pairrm(**encodings)
# logits = outputs.logits.tolist()
# comparison_results = outputs.logits > 0
# print(logits)
# # [1.9003021717071533, -1.2547134160995483]
# print(comparison_results)